BANK BETA PROJECT
--
Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Haremos predicciones de si un cliente dejará el banco pronto. Tenemos los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crearemos un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitaremos un valor F1 de al menos 0.59. Verificaremos F1 para el conjunto de prueba. 

Además, mediremos la métrica AUC-ROC y compararemos con el valor F1.

In [4]:
#IMPORTAR LIBRERIAS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [3]:
#LEER EL DATAFRAME Y VER SU INFORMACION
data = pd.read_csv('dataset/Churn.csv')
data.info()
display(data.dtypes)
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure             float64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [4]:
#REVISION DE DATOS.

geography = data['Geography'].unique()
female = data[data['Gender'] == 'Female']['Gender'].count()
male = data[data['Gender'] == 'Male']['Gender'].count()
exited = data.groupby('Exited')['CustomerId'].count()

print(f'Tenemos clientes en las siguientes regiones : {geography}')
print(f'Tenemos {female} mujes y {male} hombres como clientes')
print(f'La cantidad de clientes que han salido son: {exited[1]}, aun tenemos {exited[0]}')

Tenemos clientes en las siguientes regiones : ['France' 'Spain' 'Germany']
Tenemos 4543 mujes y 5457 hombres como clientes
La cantidad de clientes que han salido son: 2037, aun tenemos 7963


In [9]:
print(data['Tenure'].isna().sum())
print(data['Tenure'].unique())

909
[ 2.  1.  8.  7.  4.  6.  3. 10.  5.  9.  0. nan]


Aplicar metodo get_dummies en las columnas Geography y Gender, pero excluir Surname ya que no parece tener informacion reveladora para el analisis.
Podemos quedarnos con el CustomerId para identificar a los clientes sin afectar su individualidad, solo evitamos tener columnas dummies extras sin un valor importante para la codificacion de los datos.

Creacion de los 3 conjuntos de datos, entrenamiento, validacion y prueba para la enseñar y poner en marcha el modelo de prediccion para identificar cuando un cliente se irá o no. 

In [10]:
#CREAR COLUMNAS DUMMIES PARA LAS COLUMNAS CATEGORICAS.

dummies = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first= True)
print(dummies.columns)
display(dummies.head(10))

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Geography_Germany', 'Geography_Spain',
       'Gender_Male'],
      dtype='object')


,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,Hargrave,619,42,2.0,0.00,1,1,1,101348.88,1,False,False,False
1,2,15647311,Hill,608,41,1.0,83807.86,1,0,1,112542.58,0,False,True,False
2,3,15619304,Onio,502,42,8.0,159660.80,3,1,0,113931.57,1,False,False,False
3,4,15701354,Boni,699,39,1.0,0.00,2,0,0,93826.63,0,False,False,False
4,5,15737888,Mitchell,850,43,2.0,125510.82,1,1,1,79084.10,0,False,True,False
5,6,15574012,Chu,645,44,8.0,113755.78,2,1,0,149756.71,1,False,True,True
6,7,15592531,Bartlett,822,50,7.0,0.00,2,1,1,10062.80,0,False,False,True
7,8,15656148,Obinna,376,29,4.0,115046.74,4,1,0,119346.88,1,True,False,False
8,9,15792365,He,501,44,4.0,142051.07,2,0,1,74940.50,0,False,False,True
9,10,15592389,H?,684,27,2.0,134603.88,1,1,1,71725.73,0,False,False,True


In [17]:
features = dummies.drop(['Exited','Surname','RowNumber'],axis= 1)
target = dummies['Exited']
features_train , features_valid_test , target_train , target_valid_test = train_test_split(features, target, test_size =0.4 , random_state = 12345)
features_valid , features_test , target_valid , target_test = train_test_split(features_valid_test, target_valid_test, test_size =0.5 , random_state = 12345)

print('Nuestro conjunto de caracteristicas para entrenamiento tiene un cantidad de valores: ',features_train.shape)
print('Nuestro conjunto de objetivo para entrenamiento tiene un cantidad de valores: ',target_train.shape)
print('\nNuestro conjunto de caracteristicas para validacion tiene un cantidad de valores: ',features_valid.shape)
print('Nuestro conjunto de objetivo para validacion tiene un cantidad de valores: ',target_valid.shape)
print('\nNuestro conjunto de caracteristicas para prueba tiene un cantidad de valores: ',features_test.shape)
print('Nuestro conjunto de objetivo para prueba tiene un cantidad de valores: ',target_test.shape)

display(features_train.head())

Nuestro conjunto de caracteristicas para entrenamiento tiene un cantidad de valores:  (6000, 12)
Nuestro conjunto de objetivo para entrenamiento tiene un cantidad de valores:  (6000,)

Nuestro conjunto de caracteristicas para validacion tiene un cantidad de valores:  (2000, 12)
Nuestro conjunto de objetivo para validacion tiene un cantidad de valores:  (2000,)

Nuestro conjunto de caracteristicas para prueba tiene un cantidad de valores:  (2000, 12)
Nuestro conjunto de objetivo para prueba tiene un cantidad de valores:  (2000,)


,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7479,15671987,567,35,8.0,153137.74,1,1,0,88659.07,False,True,True
3411,15815628,711,37,8.0,113899.92,1,0,0,80215.20,False,False,False
6027,15799494,850,44,3.0,140393.65,2,0,1,186285.52,True,False,True
1247,15711288,512,24,6.0,0.00,2,1,0,37654.31,False,False,True
3716,15699492,665,27,2.0,147435.96,1,0,0,187508.06,True,False,False


In [21]:
numerics = ['CreditScore', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numerics])

features_train[numerics] = scaler.transform(features_train[numerics])
features_valid[numerics] = scaler.transform(features_valid[numerics])
features_test[numerics] = scaler.transform(features_test[numerics])

display(features_train.head())

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7479,-0.268372,-0.886751,-0.373192,1.030263,1.232271,-0.891560,0.642466,-1.055187,-0.187705,False,True,True
3411,1.725540,0.608663,-0.183385,1.030263,0.600563,-0.891560,-1.556504,-1.055187,-0.333945,False,False,False
6027,1.501580,2.052152,0.480939,-0.703821,1.027098,0.830152,-1.556504,0.947699,1.503095,True,False,True
1247,0.277174,-1.457915,-1.417129,0.336629,-1.233163,0.830152,0.642466,-1.055187,-1.071061,False,False,True
3716,0.113431,0.130961,-1.132419,-1.050638,1.140475,-0.891560,-1.556504,-1.055187,1.524268,True,False,False


In [ ]:
#ARBOL DE DECISION.
for depth in range(0,10,2):
    model_tree = DecisionTreeClassifier(max_depth= depth)
    